<a href="https://colab.research.google.com/github/sazzad15-1779/Project/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn
import nltk
import pickle
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np 
import tflearn
import tensorflow
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

import json
intents = json.loads(open('/content/drive/MyDrive/Colab Notebooks/intentions.json').read())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words. extend(word_list)
        documents.append(((word_list), intent["tag"]))        
        if intent['tag'] not in classes:
          classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

training = []
output_empty = [0] *len(classes)
for document in documents :
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words :
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag,output_row])
random.shuffle(training)
training= np.array(training)

train_x = list (training[:,0])
train_y = list (training[:,1])
x = Sequential()
x.add(Dense(128,input_shape = (len(train_x[0]),),activation='relu'))
x.add(Dropout(rate=.5))
x.add(Dense(64,activation='relu'))
x.add(Dropout(rate=.5))
x.add(Dense(len(train_y[0]),activation='softmax'))
sgd = SGD(learning_rate=.01,momentum=.9,nesterov=True)
x.compile(loss='categorical_crossentropy',optimizer = sgd,metrics = ['accuracy'])
model = x.fit(np.array(train_x),np.array(train_y),epochs =200,batch_size=5,verbose=1)
x.save('chatterbot.model',model)
print('Done')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Train on 102 samples
Epoch 1/200
102/102 [==============================] - 0s 1ms/sample - loss: 3.4960 - acc: 0.0196
Epoch 2/200
102/102 [==============================] - 0s 357us/sample - loss: 3.3771 - acc: 0.0490
Epoch 3/200
102/102 [==============================] - 0s 295us/sample - loss: 3.2286 - acc: 0.1569
Epoch 4/200
102/102 [==============================] - 0s 309us/sample - loss: 3.1546 - acc: 0.1863
Epoch 5/200
102/102 [==============================] - 0s 301us/sample - loss: 2.9918 - acc: 0.2059
Epoch 6/200
102/102 [==============================] - 0s 322us/sample - loss: 2.8874 - acc: 0.1961
Epoch 7/200
102/102 [==============================] - 0s 320us/sample - loss: 2.5696 - acc: 0.3235
Epoch 8/200
102/102 [==============================] - 0s 430us/sample - loss: 2.3878 - acc: 0.3824
Epoch 9/200
102/102 [==============================] - 0s 365us/sample - loss: 2.1657 - acc: 0.3922
Epoch 10/200
102/102 [==============================] - 0s 333us/sample - loss: 1

In [ ]:
from tensorflow.keras.models import load_model
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatterbot.model')

def clean_up_sentence(sentence):
  sentence_words =  nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = x.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = .25
  results = [[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD ]
  results.sort(key=lambda x: x[1], reverse = True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability' : str(r[1])})
    return return_list

def get_response(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result  = random.choice(i['responses'])
      break 
  return result
print("Hey ,Bot is listening to you ")
while True:
  message = input("")
  
  ints = predict_class(message)
  res = get_response(ints,intents)
  if message.lower() == 'bye' :
    print("bot >:",res) 
    break
  print("bot >:",res) 


Hey ,Bot is listening to you 
hello


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


bot >: Hello, thanks for visiting
hello
bot >: Hello, thanks for visiting
hello
bot >: I am fine what 
bye
bot >: Have a nice day


In [ ]:
from tensorflow.keras.models import load_model
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatterbot.model')

def clean_up_sentence(sentence):
  sentence_words =  nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = x.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = .25
  results = [[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD ]
  results.sort(key=lambda x: x[1], reverse = True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability' : str(r[1])})
    return return_list

def get_response(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result  = random.choice(i['responses'])
      break 
  return result
print("Hey ,Bot is listening to you ")
while True:
  message = input("")
  
  ints = predict_class(message)
  res = get_response(ints,intents)
  if message.lower() == 'bye' :
    print("bot >:",res) 
    break
  print("bot >:",res) 


Hey ,Bot is listening to you 
hello
bot >: I am fine what 
hello
bot >: Hello
how are you ?
bot >: All great
who are you ?
bot >: I'm Bot created by sazzad 
i need some information. do you have any bus for chittagong ?
bot >: let me see the schedule.Yes
can you tell me the bus schedule for today ?
bot >: yes,we have two schedules for day at 14.30 and another one at 17.00 and one schedule for night at 22.30.
what is your bus facility ?
bot >: yes sir,but i have to know which class you prefer?. we have three class and all service based on the class.
can you tell me the class name ?
bot >: We have business class ,economy class and normal class.
which services i got in business class?
bot >: Ok sir ,for business class we give some facility like as Extra Luxuries So You Arrive Refreshed , Complimentary Food, Drinks & Entertainment and also Additional Baggage Allowance & More Space.
ok how much cost for business class ticket ?
bot >: It's only 1200 taka
what is your economy facility?
bot >: 